# 0515

## 专利检索
1 一种基于JavaCV的自有设备网络监控视频的播放方法及系统  
2 CN202210754971 一种基于国标GB28181的推流web播放方法和系统  
3 CN202211328865 一种视频流推送系统  
4 CN202210736186 一种用于无人机的果园果实在线检测与计数方法  

## RTC在服务器的测试

[meeting-h5](https://gitee.com/nmeeting/meeting-h5/repository/blazearchive/master.zip)  
[](https://gitee.com/nmeeting/meeting-server/repository/blazearchive/master.zip?Expires=1681785197&Signature=VoIPmPHnAX3xMbK%2BzV5b8rrEbESgGADCwbiAWZsRTr8%3D)  
[](https://gitee.com/nmeeting/ion-sfu/repository/blazearchive/master.zip?Expires=1681785251&Signature=JRyD9XghvWUFhZnqEQeJDXj6X407Vkw0uqp4%2BxnPyoU%3D)

## 验证网络延迟模拟工具模拟延迟的机制
由于使用clumsy模拟网络延迟时，视频流会出现严重卡顿，经过测试此现象与采集端和推流端无关，因此怀疑该工具对一个视频帧拆分的多个tcp数据报进行延迟时，导致了一个延迟累计问题。所以编写一个程序多次发送tcp数据包并对其进行模拟延迟，抓包观察是否存在延迟累计现象。

In [2]:
import socket  			 				#导入socket模块
import random
import datetime

def server():
    host ='127.0.0.1'
    port = 8080
    web = socket.socket()
    web.bind((host,port))
    web.listen(5)
    print('waiting for client to connect ...')
    conn, addr = web.accept()
    while True :
        data = conn.recv(1024)
        curr_time = datetime.datetime.now()
        print(curr_time.second,'\t',curr_time.microsecond,'server receive\n' )

        msg = input('msg?')
        conn.sendall(msg.encode())
        curr_time = datetime.datetime.now()
        print(curr_time.second,'\t',curr_time.microsecond,'server send\n' )

        if(msg == 'exit') :
            conn.close()
            break
    print('sever exited')

def client() :
    s = socket.socket()						#创建套接字
    host = '127.0.0.1'						#IP
    port = 8080								#端口
    s.connect((host,port))					#主动初始化TCP服务器连接
    send_data = str(random.getrandbits(1024*8))  # 生成1024字节数据
    while(send_data != 'exit'):
        s.send(send_data.encode())					#发送TCP数据
        curr_time = datetime.datetime.now()
        print(curr_time.second,'\t',curr_time.microsecond,'client send\n')

         #接受对方发送过来的数据，最大接受1024字节
        recvData = s.recv(1024).decode()
        curr_time = datetime.datetime.now()
        print(curr_time.second,'\t',curr_time.microsecond,'client receive\n')

        if recvData == 'exit':
            break
        send_data = str(random.getrandbits(1024*8))
    #关闭套接字
    s.close()
    print("client exited")

In [4]:
import threading
server_thread = threading.Thread(target=server, args=[])
client_thread = threading.Thread(target=client, args=[])
server_thread.start()
client_thread.start()

server_thread.join()
client_thread.join()

waiting for client to connect ...
26 	 291576 client send

26 	 442623 server receive

32 	 579077 server send

32 	 579940 server receive

32 	 730934 client receive

32 	 731932 client send

57 	 719961 server send

57 	 720963 server receive

57 	 871111 client receive

57 	 873013 client send

20 	 28879 server send

sever exited
20 	 180868 client receive

client exited
